In [16]:
#Imports
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [17]:
#Search for target protein
target = new_client.target
target_query = target.search('influenza')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],unidentified influenza virus,unidentified influenza virus,12.0,False,CHEMBL613128,[],ORGANISM,11309
1,[],Influenza B virus,Influenza B virus,12.0,False,CHEMBL613129,[],ORGANISM,11520
2,[],Influenza A virus,Influenza A virus,12.0,False,CHEMBL613740,[],ORGANISM,11320
3,[],Influenza C virus,Influenza C virus,12.0,False,CHEMBL612783,[],ORGANISM,11552
4,"[{'xref_id': 'P03438', 'xref_name': None, 'xre...",Influenza A virus (strain A/X-31 H3N2),Influenza A virus Hemagglutinin,10.0,False,CHEMBL4918,"[{'accession': 'P03438', 'component_descriptio...",SINGLE PROTEIN,132504
5,[],Influenza A virus (H5N1),Influenza A virus (H5N1),10.0,False,CHEMBL613845,[],ORGANISM,102793
6,[],Influenza A virus H3N2,Influenza A virus H3N2,10.0,False,CHEMBL2366902,[],ORGANISM,41857
7,[],Unidentified Influenza A virus (H1N2),Unidentified Influenza A virus (H1N2),9.0,False,CHEMBL2367089,[],ORGANISM,1323429
8,"[{'xref_id': 'P63231', 'xref_name': None, 'xre...",Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,8.0,False,CHEMBL2052,"[{'accession': 'P0DOF8', 'component_descriptio...",SINGLE PROTEIN,381517
9,[],Influenza B virus (B/Lee/40),Influenza B virus (B/Lee/40),8.0,False,CHEMBL612452,[],ORGANISM,107412


Assign the eigth entry, Influenza A virus Matrix protein M2, to the ***selected_target*** variable and retrieve bioactivity data thats reported as IC<sub>50</sub> values in nM unit.

In [18]:
selected_target = targets.target_chembl_id[8]
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')
df = pd.DataFrame.from_dict(res)
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,14730724,[],CHEMBL3291757,Inhibition of wild type Influenza A virus (A/u...,B,None,None,BAO_0000190,BAO_0000249,...,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,None,None,IC50,uM,UO_0000065,None,2.92
1,None,14730725,[],CHEMBL3291757,Inhibition of wild type Influenza A virus (A/u...,B,None,None,BAO_0000190,BAO_0000249,...,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,None,None,IC50,uM,UO_0000065,None,1.5
2,None,14730726,[],CHEMBL3291757,Inhibition of wild type Influenza A virus (A/u...,B,None,None,BAO_0000190,BAO_0000249,...,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,None,None,IC50,uM,UO_0000065,None,3.38
3,None,14730727,[],CHEMBL3291757,Inhibition of wild type Influenza A virus (A/u...,B,None,None,BAO_0000190,BAO_0000249,...,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,None,None,IC50,uM,UO_0000065,None,1.64
4,None,14730730,[],CHEMBL3291757,Inhibition of wild type Influenza A virus (A/u...,B,None,None,BAO_0000190,BAO_0000249,...,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,None,None,IC50,uM,UO_0000065,None,5.54


## Datapreprocessing

In [34]:
#Drop missing data
df = df[df.standard_value.notna()]

#Label compounds as active, inactive or intermediate
bioactivity_class = []
for i in df.standard_value:
    if float(i) >= 100000:
        bioactivity_class.append('inactive')
    elif float(i) <= 1000:
        bioactivity_class.append('active')
    else:
        bioactivity_class.append('intermediate')

#Create dataframe of compund labels and:
#molecule_chembl_id
#canonical_smiles
#standard_value
data_targets = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df2 = df[data_targets]
df2 = pd.concat([df2, pd.Series(bioactivity_class, name='bioactivity_class')], axis=1)
df2.to_csv('bioactivity_data.csv', index=False)
df2

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL3286970,C1=CC2C3CNCC3C1C1CC21.Cl,2920.0,intermediate
1,CHEMBL3286971,Cl.N=C(N)N1CC2C3C=CC(C4CC34)C2C1,1500.0,intermediate
2,CHEMBL3286972,C1CC2C3CNCC3C1C1CC21.Cl,3380.0,intermediate
3,CHEMBL3286973,Cl.N=C(N)N1CC2C3CCC(C4CC34)C2C1,1640.0,intermediate
4,CHEMBL3286976,C1=CC2C1C1C=CC2C2CNCC12.Cl,5540.0,intermediate
5,CHEMBL3286977,C1CC2C1C1CCC2C2CNCC12.Cl,1240.0,intermediate
6,CHEMBL3286403,Cl.N=C(N)N1CC2C3C=CC(C4C=CC43)C2C1,4930.0,intermediate
7,CHEMBL3286978,Cl.N=C(N)N1CC2C3CCC(C4CCC43)C2C1,2080.0,intermediate
8,CHEMBL3286979,CC12CC34CNCC35CC1(C)C(C)(C5)C2(C)C4.Cl,18000.0,intermediate
9,CHEMBL3286980,CC12CC34CN(C(=N)N)CC35CC1(C)C(C)(C5)C2(C)C4.Cl,10700.0,intermediate
